In [22]:
#Kathrin Fischer
import healpy as hp
from healpy.newvisufunc import projview, newprojplot
import numpy as np
import matplotlib.pyplot as plt
import yt
from unyt import cm, s, erg, sr
import pyneb as pn

#import argparse
from tqdm import tqdm
import pickle

from astropy import units as u, constants  as cc

pc   = cc.pc.cgs.value
kB   = cc.k_B.cgs.value
Msun = cc.M_sun.cgs.value
G    = cc.G.cgs.value
Myr  = u.Myr.in_units("s")
eV = 1.60218e-12   # eV in erg

        Use to() instead. [warnings]


In [23]:
def load_CIE_data():
    # Collisional ionisation equilibrium (CIE) abundances from Gnat & Sternberg (2007, ApJS, 168, 213)
    CIE_data = np.loadtxt("CIE-abundances-gs07.txt")
    return CIE_data

def get_ion_abundance(CIE_data, ion, T):
    ion_list = dict([('HI',  1),  ('HII',  2), 
                     ('HeI', 3),  ('HeII', 4),  ('HeIII', 5),
                     ('NI',  13), ('NII',  14), 
                     ('OI',  21), ('OII',  22), ('OIII',  23), 
                     ('SI',  69), ('SII',  70), ('SIII',  71)])
    
    ion_index = ion_list[ion]
    ion_temps = data[:,0]
    T_index = np.searchsorted(ion_temps, T)
    abundance = np.zeros(T.size)
    dT = np.zeros(T.size)

    # If the requested temperature is outside of the tabulated range, we return the first or last value from the table, as appropriate
    # Otherwise, we calculate the value using linear interpolation
    
    mask1 = (T_index >= ion_temps.size)
    mask2 = (T_index == 0)
    mask3 = (~mask1 & ~mask2)

    abundance[mask1] = data[-1, ion_index]
    abundance[mask2] = data[0,  ion_index]
    dT[mask3] = ion_temps[T_index[mask3]] - ion_temps[T_index[mask3]-1]
    abundance[mask3] = data[T_index[mask3]-1,ion_index] + (data[T_index[mask3],ion_index] - data[T_index[mask3]-1,ion_index]) * ((T[mask3] - ion_temps[T_index[mask3]-1]) / dT[mask3])
    
    return abundance

def compute_emissivity(CIE_data, ion, transition, density, T):
    # Current we assume we're dealing with solar metallicty gas. Could be modified to take individual elemental
    # abundances or total metallicity as an input parameter, but that doesn't seem necessary right now
    solar_abundances = dict([('H', 1.0), ('He', 0.1), ('N', 6.03e-5), ('O', 4.57e-4), ('S', 1.38e-5)])

    # Check whether the transition wavelength we've passed in is one we're expecting to see
    # Note: if the value is a long way off from what it expects, PyNeb will also produce an error
    known_transitions = (3726, 3729, 5007, 6300, 6583, 6717, 6731, 9069, 9531) 
    if transition not in known_transitions:
        print("Unexpected transition wavelength: ", transition)
    
    if ion[0:2] == 'He':
        element = 'He'
        charge_state = len(ion) - 2
    else:
        element = ion[0]
        charge_state = len(ion) - 1  

    # Get number density of emitting ion
    elemental_abundance = solar_abundances[element]
    ion_fraction = get_ion_abundance(data, ion, T)
    ion_density = elemental_abundance * ion_fraction * density

    # Get electron number density. Currently, we also assume CIE here but we could alternatively extract this from the FLASH
    # simulation snapshots and pass it in as an additional parameter.
    #
    # Note: we include contribution from helium here, but not from metals -- this is a <1% effect, hence unlikely to be important
    electron_fraction = get_ion_abundance(data, 'HII', T) + 0.1 * get_ion_abundance(data, 'HeII', T) + 0.2 * get_ion_abundance(data, 'HeIII', T)
    electron_density = electron_fraction * density

    atom = pn.Atom(element, charge_state)
    
    # This computes the emission coefficient (units: erg cm^3 s^-1) for the transition from the specified ion at the specified wavelength
    # To compute the emissivity (units: erg cm^-3 s^-1), we then simply multiply by the ion density and the electron density
    emission_coeff = atom.getEmissivity(tem=T, den=density, wave=transition)
    emissivity = emission_coeff * ion_density * electron_density
    
    return emissivity


In [ ]:
# --- Setup ---
file_list = ["data/SILCC_hdf5_plt_cnt_1080"]
names = ["10.8Myr"]
nside = 64
radius = 80  # pc
odir = "."
radius_str = str(int(radius)).zfill(4)
file_suffix = f"r{radius_str}_CIE"
cx, cy, cz = -80., -150., 0.0
c = [cx*pc, cy*pc, cz*pc]

# --- Define transitions ---
transitions = {
    "OI_6300":    ("OI", 6300),
    "OII_3726":   ("OII", 3726),
    "OII_3729":   ("OII", 3729),
    "OIII_5007":  ("OIII", 5007),
    "NII_6583":   ("NII", 6583),
    "SII_6717":   ("SII", 6717),
    "SII_6731":   ("SII", 6731),
    "SIII_9069":  ("SIII", 9069),
    "SIII_9531":  ("SIII", 9531),
}

data = load_CIE_data()  # load once

# Dictionary to hold maps
emission_maps = {}

for files, name in zip(file_list, names):
    print(f"Processing file: {files}")
    ds = yt.load(files)
    sp = ds.sphere(c, (radius, "pc"))

    # Coordinates & geometry
    Lx, Ly, Lz = ds.domain_width.in_units("pc").v
    posx_ctr = (sp[("gas", "x")].in_units("pc").v - cx + Lx/2.) % Lx - Lx/2.
    posy_ctr = (sp[("gas", "y")].in_units("pc").v - cy + Ly/2.) % Ly - Ly/2.
    posz_ctr =  sp[("gas", "z")].in_units("pc").v - cz
    rad_ctr = np.sqrt(posx_ctr**2 + posy_ctr**2 + posz_ctr**2)

    volu = sp[("gas", "cell_volume")].in_units("pc**3").v
    R = (3 * volu / (4 * np.pi))**(1./3.)
    angle = np.arctan2(R, rad_ctr)
    vec_norm_x = posx_ctr / rad_ctr
    vec_norm_y = posy_ctr / rad_ctr
    vec_norm_z = posz_ctr / rad_ctr
    NPIX = hp.nside2npix(nside)

    # Physical properties
    T = sp[("gas", "temperature")].v
    n = sp[("gas", "number_density")].v
    logT = np.log10(T)
    logn = np.log10(n)
    Tmin, Tmax = np.min(logT), np.max(logT)
    nmin, nmax = np.min(logn), np.max(logn)
    T_grid = np.logspace(Tmin, Tmax, 100)
    n_grid = np.logspace(nmin, nmax, 100)
    logT_grid = np.log10(T_grid)
    logn_grid = np.log10(n_grid)
    T_idx = np.clip(np.digitize(logT, logT_grid) - 1, 0, len(logT_grid)-1)
    n_idx = np.clip(np.digitize(logn, logn_grid) - 1, 0, len(logn_grid)-1)

    # Loop over transitions
    for label, (ion, wavelength) in transitions.items():
        print(f"Generating map for: {label}")
        em_table = compute_emissivity(data, ion=ion, transition=wavelength, density=n_grid, T=T_grid)
        em_cell = em_table[n_idx, T_idx]
        em_map = np.zeros(NPIX)
        em_map_flux = np.zeros(NPIX)

        for i in range(len(rad_ctr)):
            pix = hp.query_disc(nside, [vec_norm_x[i], vec_norm_y[i], vec_norm_z[i]], angle[i], inclusive=True)
            if em_cell[i] > 0:
                em_map[pix] += em_cell[i] * volu[i] / len(pix)
                em_map_flux[pix] += em_cell[i] * volu[i] / (4 * np.pi * (R[i]*pc)**2)

        emission_maps[label] = (files, name, em_map, em_map_flux)


yt : [INFO     ] 2025-04-16 10:50:19,706 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-04-16 10:50:19,708 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-04-16 10:50:19,712 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-04-16 10:50:19,715 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-04-16 10:50:19,718 Parameters: cosmological_simulation   = 0


Processing file: data/SILCC_hdf5_plt_cnt_1080
Generating map for: OI_6300


In [ ]:
#plots of emission and flux
for label, (files, name, em_map, em_map_flux) in emission_maps.items():
    fig = plt.figure(figsize=(10, 5))
    em_plot = np.log10(np.where(em_map > 0, em_map, np.nan))
    em_flux_plot = np.log10(np.where(em_map_flux > 0, em_map_flux, np.nan))
    
    projview(
        em_plot,
        coord=["G"],
        graticule=True,
        graticule_labels=True,
        unit=f"{label} Emissivity [log erg s⁻¹ sr⁻¹]", #stimmt das?
        xlabel="Longitude", ylabel="Latitude",
        cb_orientation="vertical",
        min=-28, max=-24,
        projection_type="mollweide",
        title=f"{label} Emission — {name}",
        flip="geo"
    )
    plt.savefig(f"{odir}/{files}-{file_suffix}-{label}.pdf", bbox_inches="tight")

    projview(
        em_flux_plot,
        coord=["G"],
        graticule=True,
        graticule_labels=True,
        unit=f"{label} Emission flux [log erg cm⁻² s⁻¹]",
        xlabel="Longitude", ylabel="Latitude",
        cb_orientation="vertical",
        min=-66, max=-60,
        projection_type="mollweide",
        title=f"{label} Emission flux — {name}",
        flip="geo"
    )
    plt.savefig(f"{odir}/{files}-f{file_suffix}-{label}.pdf", bbox_inches="tight")
    plt.close()


In [12]:
# --- Setup ---
file_list = ["data/SILCC_hdf5_plt_cnt_1080"]
names = ["10.8Myr"]
nside = 64
radius = 80  # pc
odir = "."
radius_str = str(int(radius)).zfill(4)
file_suffix = f"r{radius_str}_CIE"
cx, cy, cz = -80., -150., 0.0
c = [cx*pc, cy*pc, cz*pc]

transitions = {
    "OI_6300":    ("OI", 6300),
    "OII_3726":   ("OII", 3726),
    "OII_3729":   ("OII", 3729),
    "OIII_5007":  ("OIII", 5007),
    "NII_6583":   ("NII", 6583),
    "SII_6717":   ("SII", 6717),
    "SII_6731":   ("SII", 6731),
    "SIII_9069":  ("SIII", 9069),
    "SIII_9531":  ("SIII", 9531),}

# --- Define the transition you want to project ---
label = "OIII_5007"
ion, wavelength = ("OIII", 5007)

# --- Load CIE Data and Precompute Emissivity Table ---
data = load_CIE_data()

# Loop over all files
for files, name in zip(file_list, names):
    print(f"Processing file: {files}")
    ds = yt.load(files)
    sp = ds.sphere(c, (radius, "pc"))

    # Position calculations
    Lx, Ly, Lz = ds.domain_width.in_units("pc").v
    posx_ctr = (sp[("gas", "x")].in_units("pc").v - cx + Lx/2.) % Lx - Lx/2.
    posy_ctr = (sp[("gas", "y")].in_units("pc").v - cy + Ly/2.) % Ly - Ly/2.
    posz_ctr =  sp[("gas", "z")].in_units("pc").v - cz
    rad_ctr = np.sqrt(posx_ctr**2 + posy_ctr**2 + posz_ctr**2)

    # Healpix geometry
    volu = sp[("gas", "cell_volume")].in_units("pc**3").v
    R = (3 * volu / (4 * np.pi))**(1./3.)
    angle = np.arctan2(R, rad_ctr)
    vec_norm_x = posx_ctr / rad_ctr
    vec_norm_y = posy_ctr / rad_ctr
    vec_norm_z = posz_ctr / rad_ctr
    NPIX = hp.nside2npix(nside)
    em_map = np.zeros(NPIX)

    # Temperature and density
    T = sp[("gas", "temperature")].v
    n = sp[("gas", "number_density")].v

    # Generate lookup table grid
    logT = np.log10(T)
    logn = np.log10(n)
    Tmin, Tmax = np.min(logT), np.max(logT)
    nmin, nmax = np.min(logn), np.max(logn)
    T_grid = np.logspace(Tmin, Tmax, 100)
    n_grid = np.logspace(nmin, nmax, 100)
    logT_grid = np.log10(T_grid)
    logn_grid = np.log10(n_grid)

    # Precompute 2D emissivity grid
    emissivity_table = compute_emissivity(cie_data, ion=ion, transition=wavelength, density=n_grid, T=T_grid)

    # Digitize and assign emissivity to cells
    T_idx = np.clip(np.digitize(logT, logT_grid) - 1, 0, len(logT_grid)-1)
    n_idx = np.clip(np.digitize(logn, logn_grid) - 1, 0, len(logn_grid)-1)
    em_cell = emissivity_table[n_idx, T_idx]

    # Build map
    for i in tqdm(range(len(rad_ctr))):
        pix = hp.query_disc(nside, [vec_norm_x[i], vec_norm_y[i], vec_norm_z[i]], angle[i], inclusive=True)
        if em_cell[i] > 0:
            em_map[pix] += em_cell[i] * volu[i] / len(pix)

    # Save one Mollweide projection
    fig = plt.figure(figsize=(10, 5))
    em_plot = np.log10(np.where(em_map > 0, em_map, np.nan))
    projview(em_plot, coord=["G"], graticule=True, graticule_labels=True,
             unit=f"{label} Emissivity [log erg cm⁻³ s⁻¹ sr⁻¹]",
             xlabel="Longitude", ylabel="Latitude",
             cb_orientation="vertical", min=-32, max=-25,
             projection_type="mollweide", title=f"{label} Emission — {name}", flip="geo")
    plt.savefig(f"{odir}/{files}-{file_suffix}-{label}.pdf", bbox_inches="tight")
    plt.close()


yt : [INFO     ] 2025-04-10 08:22:37,156 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-04-10 08:22:37,158 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-04-10 08:22:37,158 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-04-10 08:22:37,161 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-04-10 08:22:37,162 Parameters: cosmological_simulation   = 0


Processing file: data/SILCC_hdf5_plt_cnt_1080


100%|██████████| 1293308/1293308 [00:15<00:00, 81281.47it/s] 


<Figure size 1000x500 with 0 Axes>

In [9]:
#for one transition / without loop
# --- Setup ---
file_list = ["data/SILCC_hdf5_plt_cnt_1080"]
names = ["10.8Myr"]
nside = 64
radius = 80  # pc
odir = "."
radius_str = str(int(radius)).zfill(4)
file_suffix = f"r{radius_str}_CIE"
cx, cy, cz = -80., -150., 0.0
c = [cx*pc, cy*pc, cz*pc]

# --- Define the transition you want to project ---
label = "OIII_5007"
ion, wavelength = ("OIII", 5007)

# --- Load CIE Data and Precompute Emissivity Table ---
data = load_CIE_data()

# Loop over all files
for files, name in zip(file_list, names):
    print(f"Processing file: {files}")
    ds = yt.load(files)
    sp = ds.sphere(c, (radius, "pc"))

    # Position calculations
    Lx, Ly, Lz = ds.domain_width.in_units("pc").v
    posx_ctr = (sp[("gas", "x")].in_units("pc").v - cx + Lx/2.) % Lx - Lx/2.
    posy_ctr = (sp[("gas", "y")].in_units("pc").v - cy + Ly/2.) % Ly - Ly/2.
    posz_ctr =  sp[("gas", "z")].in_units("pc").v - cz
    rad_ctr = np.sqrt(posx_ctr**2 + posy_ctr**2 + posz_ctr**2)

    # Healpix geometry
    volu = sp[("gas", "cell_volume")].in_units("pc**3").v
    R = (3 * volu / (4 * np.pi))**(1./3.)
    angle = np.arctan2(R, rad_ctr)
    vec_norm_x = posx_ctr / rad_ctr
    vec_norm_y = posy_ctr / rad_ctr
    vec_norm_z = posz_ctr / rad_ctr
    NPIX = hp.nside2npix(nside)
    em_map = np.zeros(NPIX)

    # Temperature and density
    T = sp[("gas", "temperature")].v
    n = sp[("gas", "number_density")].v

    # Generate lookup table grid
    logT = np.log10(T)
    logn = np.log10(n)
    Tmin, Tmax = np.min(logT), np.max(logT)
    nmin, nmax = np.min(logn), np.max(logn)
    T_grid = np.logspace(Tmin, Tmax, 100)
    n_grid = np.logspace(nmin, nmax, 100)
    logT_grid = np.log10(T_grid)
    logn_grid = np.log10(n_grid)

    # Precompute 2D emissivity grid
    emissivity_table = compute_emissivity(cie_data, ion=ion, transition=wavelength, density=n_grid, T=T_grid)

    # Digitize and assign emissivity to cells
    T_idx = np.clip(np.digitize(logT, logT_grid) - 1, 0, len(logT_grid)-1)
    n_idx = np.clip(np.digitize(logn, logn_grid) - 1, 0, len(logn_grid)-1)
    em_cell = emissivity_table[n_idx, T_idx]

    # Build map
    for i in tqdm(range(len(rad_ctr))):
        pix = hp.query_disc(nside, [vec_norm_x[i], vec_norm_y[i], vec_norm_z[i]], angle[i], inclusive=True)
        if em_cell[i] > 0:
            em_map[pix] += em_cell[i] * volu[i] / len(pix)

    # Save one Mollweide projection
    fig = plt.figure(figsize=(10, 5))
    em_plot = np.log10(np.where(em_map > 0, em_map, np.nan))
    projview(em_plot, coord=["G"], graticule=True, graticule_labels=True,
             unit=f"{label} Emissivity [log erg cm⁻³ s⁻¹ sr⁻¹]",
             xlabel="Longitude", ylabel="Latitude",
             cb_orientation="vertical", min=-32, max=-25,
             projection_type="mollweide", title=f"{label} Emission — {name}", flip="geo")
    plt.savefig(f"{odir}/{files}-{file_suffix}-{label}.pdf", bbox_inches="tight")
    plt.close()


yt : [INFO     ] 2025-04-09 15:54:24,508 Parameters: current_time              = 340597785179995.1
yt : [INFO     ] 2025-04-09 15:54:24,510 Parameters: domain_dimensions         = [8 8 8]
yt : [INFO     ] 2025-04-09 15:54:24,511 Parameters: domain_left_edge          = [-7.715e+20 -7.715e+20 -7.715e+20]
yt : [INFO     ] 2025-04-09 15:54:24,511 Parameters: domain_right_edge         = [7.715e+20 7.715e+20 7.715e+20]
yt : [INFO     ] 2025-04-09 15:54:24,512 Parameters: cosmological_simulation   = 0


Processing file: data/SILCC_hdf5_plt_cnt_1080


100%|██████████| 1293308/1293308 [00:15<00:00, 85707.75it/s] 


<Figure size 1000x500 with 0 Axes>